<b>Lab | Handling Data Imbalance in Classification Models <br>
    
For this lab and in the next lessons we will build a model on customer churn binary classification problem. You will be using files_for_lab/Customer-Churn.csv file.

Scenario
You are working as an analyst with this internet service provider. You are provided with this historical data about your company's customers and their churn trends. Your task is to build a machine learning model that will help the company identify customers that are more likely to default/churn and thus prevent losses from such customers.

Instructions
In this lab, we will first take a look at the degree of imbalance in the data and correct it using the techniques we learned on the class.

Here is the list of steps to be followed (building a simple model without balancing the data):

- Import the required libraries and modules that you would need.<br>
- Read that data into Python and call the dataframe churnData.<br>
- Check the datatypes of all the columns in the data. You would see that the column TotalCharges is object type. Convert this column into numeric type using pd.to_numeric function.<br>
- Check for null values in the dataframe. Replace the null values.<br>
- Use the following features: tenure, SeniorCitizen, MonthlyCharges and TotalCharges:<br>
-- Scale the features either by using normalizer or a standard scaler.<br>
-- Split the data into a training set and a test set.<br>
-- Fit a logistic regression model on the training data.<br>
-- Check the accuracy on the test data.<br>
Note: So far we have not balanced the data.<br>

Managing imbalance in the dataset

Check for the imbalance.
Use the resampling strategies used in class for upsampling and downsampling to create a balance between the two classes.
Each time fit the model and see how the accuracy of the model is.

In [31]:
# import libraries

import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks


import warnings
warnings.filterwarnings('ignore')

In [2]:
#read the data into Python

churnData = pd.read_csv("files_for_lab/Customer-Churn.csv")
churnData.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85,No
1,Male,0,No,No,34,Yes,Yes,No,Yes,No,No,No,One year,56.95,1889.5,No
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15,Yes
3,Male,0,No,No,45,No,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65,Yes


In [3]:
# how many rows and columns do we have?

churnData.shape

(7043, 16)

In [4]:
# let's standardise the column names to lower case 

churnData.columns = churnData.columns.str.lower()
churnData.head()

,gender,seniorcitizen,partner,dependents,tenure,phoneservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,monthlycharges,totalcharges,churn
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85,No
1,Male,0,No,No,34,Yes,Yes,No,Yes,No,No,No,One year,56.95,1889.5,No
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15,Yes
3,Male,0,No,No,45,No,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65,Yes


In [5]:
# check the datatypes 

churnData.dtypes

gender               object
seniorcitizen         int64
partner              object
dependents           object
tenure                int64
phoneservice         object
onlinesecurity       object
onlinebackup         object
deviceprotection     object
techsupport          object
streamingtv          object
streamingmovies      object
contract             object
monthlycharges      float64
totalcharges         object
churn                object
dtype: object

In [6]:
# change TotalCharges to numeric

churnData['totalcharges'] = pd.to_numeric(churnData['totalcharges'], errors = "coerce")

In [7]:
#check dtypes again

churnData.dtypes

gender               object
seniorcitizen         int64
partner              object
dependents           object
tenure                int64
phoneservice         object
onlinesecurity       object
onlinebackup         object
deviceprotection     object
techsupport          object
streamingtv          object
streamingmovies      object
contract             object
monthlycharges      float64
totalcharges        float64
churn                object
dtype: object

In [8]:
# get the sum of the nulls

null_counts = churnData.isnull().sum()
null_counts

gender               0
seniorcitizen        0
partner              0
dependents           0
tenure               0
phoneservice         0
onlinesecurity       0
onlinebackup         0
deviceprotection     0
techsupport          0
streamingtv          0
streamingmovies      0
contract             0
monthlycharges       0
totalcharges        11
churn                0
dtype: int64

In [9]:
# I chose to fill the nulls with the mean

df_full = churnData.fillna(churnData.mean())
df_full.head()

,gender,seniorcitizen,partner,dependents,tenure,phoneservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,monthlycharges,totalcharges,churn
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85,No
1,Male,0,No,No,34,Yes,Yes,No,Yes,No,No,No,One year,56.95,1889.50,No
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15,Yes
3,Male,0,No,No,45,No,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65,Yes


In [10]:
# check again for null values

null_counts2 = df_full.isnull().sum()
null_counts2

gender              0
seniorcitizen       0
partner             0
dependents          0
tenure              0
phoneservice        0
onlinesecurity      0
onlinebackup        0
deviceprotection    0
techsupport         0
streamingtv         0
streamingmovies     0
contract            0
monthlycharges      0
totalcharges        0
churn               0
dtype: int64

In [11]:
# create new dataframe with only select columns

new_df = df_full[["tenure", "seniorcitizen", "monthlycharges", "totalcharges"]]
new_df

,tenure,seniorcitizen,monthlycharges,totalcharges
0,1,0,29.85,29.85
1,34,0,56.95,1889.50
2,2,0,53.85,108.15
3,45,0,42.30,1840.75
4,2,0,70.70,151.65
...,...,...,...,...
7038,24,0,84.80,1990.50
7039,72,0,103.20,7362.90
7040,11,0,29.60,346.45
7041,4,1,74.40,306.60


In [12]:
transformer = MinMaxScaler().fit(new_df)
X_minmax = transformer.transform(new_df)
print(X_minmax.shape)

(7043, 4)


In [13]:
X_minmax

array([[0.01388889, 0.        , 0.11542289, 0.0012751 ],
       [0.47222222, 0.        , 0.38507463, 0.21586661],
       [0.02777778, 0.        , 0.35422886, 0.01031041],
       ...,
       [0.15277778, 0.        , 0.11293532, 0.03780868],
       [0.05555556, 1.        , 0.55870647, 0.03321025],
       [0.91666667, 0.        , 0.86965174, 0.78764136]])

In [14]:
X_num_norm = pd.DataFrame(X_minmax, columns = new_df.columns)
X_num_norm.head()

,tenure,seniorcitizen,monthlycharges,totalcharges
0,0.013889,0.0,0.115423,0.001275
1,0.472222,0.0,0.385075,0.215867
2,0.027778,0.0,0.354229,0.010310
3,0.625000,0.0,0.239303,0.210241
4,0.027778,0.0,0.521891,0.015330


In [17]:
# split the data into train test split

X = X_num_norm
y = churnData["churn"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42) 

In [20]:
# apply logisitc regression

# Create a Logistic Regression model
logreg = LogisticRegression()

# Train the model on the training data
logreg.fit(X_train, y_train)

# Make predictions on the test data
y_pred = logreg.predict(X_test)

# Evaluate the model
accuracy = logreg.score(X_test, y_test)

# Print the accuracy
print("Accuracy:", accuracy)

Accuracy: 0.7989778534923339


In [21]:
# create a report 

report_dict = classification_report(y_test, y_pred, output_dict = True)
report_df = pd.DataFrame(report_dict).transpose()
report_df

,precision,recall,f1-score,support
No,0.819120,0.929017,0.870614,1282.000000
Yes,0.703583,0.450939,0.549618,479.000000
accuracy,0.798978,0.798978,0.798978,0.798978
macro avg,0.761351,0.689978,0.710116,1761.000000
weighted avg,0.787693,0.798978,0.783302,1761.000000


#### Managing Imbalanced Data

In [29]:
# How imbalanced is churn?

y.value_counts()

No     5174
Yes    1869
Name: churn, dtype: int64

In [23]:
# check for imbalance 

pd.Series(y_pred).value_counts()

No     1454
Yes     307
dtype: int64

In [24]:
y_test.value_counts()

No     1282
Yes     479
Name: churn, dtype: int64

### Downsampling

In [36]:
RUS = RandomUnderSampler()
X_RUS, y_RUS = RUS.fit_resample(X, y) 
y_RUS.value_counts()

No     1869
Yes    1869
Name: churn, dtype: int64

In [37]:
y.value_counts()

No     5174
Yes    1869
Name: churn, dtype: int64

In [38]:
X_train_RUS, X_test_RUS, y_train_RUS, y_test_RUS = train_test_split(X_RUS, y_RUS, test_size = 0.25, random_state = 42)

logreg = LogisticRegression()
logreg.fit(X_train_RUS, y_train_RUS)
y_pred = logreg.predict(X_test_RUS)
accuracy = logreg.score(X_test_RUS, y_test_RUS)
print("Accuracy:", accuracy)

Accuracy: 0.746524064171123


The accuracy decreased with downsampling

### Upsampling

In [39]:
ROS = RandomOverSampler()
X_ROS, y_ROS = ROS.fit_resample(X, y)
y_ROS.value_counts()

No     5174
Yes    5174
Name: churn, dtype: int64

In [40]:
y.value_counts()

No     5174
Yes    1869
Name: churn, dtype: int64

In [41]:
X_train_ROS, X_test_ROS, y_train_ROS, y_test_ROS = train_test_split(X_ROS, y_ROS, test_size = 0.25, random_state = 42)

logreg = LogisticRegression()
logreg.fit(X_train_ROS, y_train_ROS)
y_pred = logreg.predict(X_test_ROS)
accuracy = logreg.score(X_test_ROS, y_test_ROS)
print("Accuracy:", accuracy)

Accuracy: 0.741012756088133


Very little difference between upsampling and downsampling

### SMOTE Technique

In [ ]:
# SMOTE technique

from imblearn.over_sampling import SMOTE
smote = SMOTE()
X = churnData[['tenure', 'SeniorCitizen','MonthlyCharges', "TotalCharges"]]
transformer = StandardScaler().fit(X)
X = transformer.transform(X)
y = churnData['Churn']
X_sm, y_sm = smote.fit_resample(X, y)
y_sm.value_counts()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42) 
classing = LogisticRegression(random_state = 0, multi_class='ovr').fit(X_train, y_train) 
predictions = classing.predict(X_test)

In [ ]:
accuracy = logreg.score(X_test, y_test)

# Print the accuracy
print("Accuracy:", accuracy)